In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/train_data.csv')

## Feature Selection

In [5]:
df.head(3)

,index,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,street,...,is_weekend,daily_trans_count,is_high_risk_hour,city_size,merchant_num_trans_1_day_window,merchant_risk_1_day_window,merchant_num_trans_7_day_window,merchant_risk_7_day_window,merchant_num_trans_30_day_window,merchant_risk_30_day_window
0,0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,561 Perry Cove,...,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,43039 Riley Greens Suite 393,...,0,10,1,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,594 White Dale Suite 530,...,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.columns

Index(['index', 'Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant',
       'category', 'amt', 'first', 'last', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'trans_date', 'gender_M',
       'job_category', 'job_encoded', 'category_encoded', 'age', 'age_group',
       'distance', 'avg_distance_30_days', 'distance_over_avg_30_days',
       'sum_amount_30_days', 'count_amount_30_days', 'avg_amount_30_days',
       'amount_over_average_30_days', 'avg_distance_90_days',
       'distance_over_avg_90_days', 'sum_amount_90_days',
       'count_amount_90_days', 'avg_amount_90_days',
       'amount_over_average_90_days', 'avg_distance_180_days',
       'distance_over_avg_180_days', 'sum_amount_180_days',
       'count_amount_180_days', 'avg_amount_180_days',
       'amount_over_average_180_days', 'transaction_hour',
       'transaction_day_of_week', 'is_holiday', 'is_weekend',
   

In [10]:
drop_col = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category','first', 'last', 'street', 'city', 'state', 'zip', 'job', 'dob', 'unix_time', 'is_fraud']

In [ ]:
train = df[df['trans_date_trans_time'] < '2020-01-01']
validation = df[df['trans_date_trans_time'] >= '2020-01-01']

X_train = train.drop(drop_col, axis=1).select_dtypes(include=np.number)
y_train = train['is_fraud']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)
feature_importances = clf.feature_importances_
X.columns[np.argsort(feature_importances)[::-1]]